In [1]:
import numpy as np
import pandas as pd
import pypower.api as pp

## PF benchmark

### Manual solution

In [2]:
from pypower.api import ext2int, bustypes, makeBdc, makeSbus, dcpf, int2ext, case39

from numpy import r_, c_, ix_, zeros, pi, ones, exp, argmax, union1d
from numpy import flatnonzero as find

from pypower.idx_bus import PD, QD, VM, VA, GS, BUS_TYPE, PV, PQ, REF
from pypower.idx_brch import PF, PT, QF, QT
from pypower.idx_gen import PG, QG, VG, QMAX, QMIN, GEN_BUS, GEN_STATUS

# Read PyPower case
ppc = case39()

## add zero columns to branch for flows if needed 
if ppc["branch"].shape[1] < QT: 
    ppc["branch"] = c_[ppc["branch"], 
                    zeros((ppc["branch"].shape[0], 
                            QT - ppc["branch"].shape[1] + 1))] 

## convert to internal indexing
ppc = ext2int(ppc)
baseMVA, bus, gen, branch = \
    ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"]

## get bus index lists of each type of bus
ref, pv, pq = bustypes(bus, gen)

## generator info
on = find(gen[:, GEN_STATUS] > 0)      ## which generators are on?
gbus = gen[on, GEN_BUS].astype(int)    ## what buses are they at?

##-----  run the power flow  -----
## initial state
Va0 = bus[:, VA] * (pi / 180)

## build B matrices and phase shift injections
B, Bf, _, _ = makeBdc(baseMVA, bus, branch)

## compute complex bus power injections [generation - load]
Pbus = makeSbus(baseMVA, bus, gen).real

## "run" the power flow
pvpq = np.matrix(r_[pv, pq]) 

# initialize result vector 
# Va = np.copy(Va0)
Va = np.zeros_like(Va0)

B_dense = B.todense()
B_bar = B_dense[pvpq.T, pvpq]
P_bar = Pbus[pvpq]

## update angles for non-reference buses
Va[pvpq] = np.dot(np.linalg.inv(B_bar), P_bar.T).reshape(1,38)
 
# update data matrices with solution
branch[:, [QF, QT]] = zeros((branch.shape[0], 2))
branch[:, PF] = (Bf * Va) * baseMVA
branch[:, PT] = -branch[:, PF]
bus[:, VM] = ones(bus.shape[0])
bus[:, VA] = Va * (180 / pi)
## update Pg for slack generator (1st gen at ref bus)
## (note: other gens at ref bus are accounted for in Pbus)
##      Pg = Pinj + Pload
##      newPg = oldPg + newPinj - oldPinj
refgen = find(gbus == ref)
gen[on[refgen[0]], PG] = gen[on[refgen[0]], PG] + (B[ref, :] * Va - Pbus[ref])* baseMVA


##-----  output results  -----
## convert back to original bus numbering & print results
ppc["bus"], ppc["gen"], ppc["branch"] = bus, gen, branch
results = int2ext(ppc)
r_bus = results["bus"]
r_branch = results["branch"]
r_gen = results["gen"]

### PyPower solution

In [3]:
ppc = case39()
pp_dcpf, success = pp.rundcpf(ppc)
pp_bus = pp_dcpf["bus"]
pp_branch = pp_dcpf["branch"]
pp_gen = pp_dcpf["gen"]

PYPOWER Version 5.1.4, 27-June-2018 -- DC Power Flow

Converged in 0.00 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             39     Total Gen Capacity    7367.0           0.0 to 0.0
Generators        10     On-line Capacity      7367.0           0.0 to 0.0
Committed Gens    10     Generation (actual)   6254.2               0.0
Loads             21     Load                  6254.2               0.0
  Fixed           21       Fixed               6254.2               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches          46     Losses (I^2 * Z)         0.00              0.00
Transformers      12     Branch Charging (inj)     -                0.0
Inter-ties         6     Tota

### Comparison

In [4]:
print("Bus voltage magitude:")
print("Manual: ", r_bus[:,VM])
print("Pypower: ", pp_bus[:,VM])

print("Bus voltage angle:")
print("Manual: ", r_bus[:,VA])
print("Pypower: ", pp_bus[:,VA])

print("Real power generation:")
print("Manual: ", r_gen[:,PG])
print("PyPower: ", pp_gen[:,PG])

print("Real power flow on branch:")
print("Manual: ", r_branch[:,PF])
print("PyPower: ", pp_branch[:,PF])

Bus voltage magitude:
Manual:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Pypower:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Bus voltage angle:
Manual:  [-12.3043699   -8.10439551 -10.98912011 -11.64954416 -10.34642157
  -9.57959794 -11.94362211 -12.50942954 -13.12810348  -7.15074661
  -7.99063881  -8.05839236  -7.91227145  -9.66724476 -10.10326532
  -8.56868354  -9.72097327 -10.66384373  -3.42925212  -4.8708231
  -5.97921632  -1.09597681  -1.32272635  -8.41614329  -6.81447228
  -7.81782625  -9.97159067  -3.86996915  -0.83007595  -5.44694564
   0.           0.81909632   2.07263699   0.41545511   4.36280697
   7.40456678   0.54299335   6.77404802 -13.46108183]
Pypower:  [-12.3043699   -8.10439551 -10.98912011 -11.64954416 -10.34642157
  -9.57959794 -11.94362211 -12.50942954 -13.12810348  -7.15074661
  -7.99063881  -8.05839236  -7.912

## OPF benchmark

### Manual solution

In [5]:
import numpy as np
from numpy import flatnonzero as find
from pypower.api import case9, ext2int, bustypes, makeBdc, rundcpf, ppoption, case39
from pypower.idx_bus import BUS_TYPE, REF, VA, PD, LAM_P, LAM_Q, MU_VMAX, MU_VMIN
from pypower.idx_gen import PG, MU_PMAX, MU_PMIN, MU_QMAX, MU_QMIN, PMAX, PMIN, GEN_BUS
from pypower.idx_brch import PF, PT, QF, QT, RATE_A, MU_SF, MU_ST
from pypower.idx_cost import COST

# Run PyPower case to get initial condition
ppc = case39()
ppopt = ppoption(VERBOSE=0)
pp_dcpf, success = rundcpf(ppc, ppopt)
pp_bus = pp_dcpf["bus"]
pp_branch = pp_dcpf["branch"]
pp_gen = pp_dcpf["gen"]


Converged in 0.00 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             39     Total Gen Capacity    7367.0           0.0 to 0.0
Generators        10     On-line Capacity      7367.0           0.0 to 0.0
Committed Gens    10     Generation (actual)   6254.2               0.0
Loads             21     Load                  6254.2               0.0
  Fixed           21       Fixed               6254.2               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches          46     Losses (I^2 * Z)         0.00              0.00
Transformers      12     Branch Charging (inj)     -                0.0
Inter-ties         6     Total Inter-tie Flow   736.4               0.0
Areas     

In [6]:
## convert to internal indexing
ppc = ext2int(ppc)
baseMVA, bus, gen, branch, gencost = \
    ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"], ppc["gencost"]

## get bus index lists of each type of bus
ref, pv, pq = bustypes(bus, gen)
pvpq = np.matrix(np.r_[pv, pq])

## generator info
gbus = gen[:, GEN_BUS].astype(int)    ## what buses are they at?
refgen = find(gbus == ref)

## build B matrices and phase shift injections
B, Bf, _, _ = makeBdc(baseMVA, bus, branch)
B = B.todense()
Bf = Bf.todense()

# Problem dimensions
NG = gen.shape[0] # Number of generators
NB = bus.shape[0] # Number of buses
NBr = branch.shape[0] # Number of lines
NL = 3 # Number of loads

CG = np.zeros((NB,NG))
CG[gbus,range(NG)] = 1

# Generator capacity limit p.u.
Pmax = gen[:, PMAX]/baseMVA
Pmin = gen[:, PMIN]/baseMVA

# Line flow limit p.u.
Lmax = branch[:, RATE_A]/baseMVA
Lmin = - branch[:, RATE_A]/baseMVA

# Quadratic cost coefficients
# Convert to p.u.
Cg2 = gencost[:, COST]*baseMVA**2
Cg1 = gencost[:, COST+1]*baseMVA
Cg0 = gencost[:, COST+2]

# Generator and demand set points
# Power in p.u., Va in rad
Pg0 = pp_gen[:, PG]/baseMVA
Pd0 = pp_bus[:, PD]/baseMVA
Pinj0 = (np.matmul(CG, Pg0) - Pd0)
Va0 = pp_bus[:, VA]/180*np.pi

In [7]:
from pyomo.environ import *

model = ConcreteModel(name="OPF")

# Define sets
model.buses = Set(initialize=range(NB))
model.lines = Set(initialize=range(NBr))
model.gens = Set(initialize=range(NG))
model.loads = Set(initialize=range(NL))

# Define parameters
def Pd_init(model, i):
    return Pd0[i]
model.Pd = Param(model.buses, initialize=Pd_init)

def B_init(model, i, j):
    return B[i,j]
model.B = Param(model.buses, model.buses, initialize=B_init)

def Bf_init(model, i, j):
    return Bf[i,j]
model.Bf = Param(model.lines, model.buses, initialize=Bf_init)

def Pmax_init(model, i):
    return Pmax[i]
model.Pmax = Param(model.gens, initialize=Pmax_init)

def Pmin_init(model, i):
    return Pmin[i]
model.Pmin = Param(model.gens, initialize=Pmin_init)

def Lmax_init(model, i):
    return Lmax[i]
model.Lmax = Param(model.lines, initialize=Lmax_init)

def Lmin_init(model, i):
    return Lmin[i]
model.Lmin = Param(model.lines, initialize=Lmin_init)

def Cg0_init(model, i):
    return Cg0[i]
model.Cg0 = Param(model.gens, initialize=Cg0_init)

def Cg1_init(model, i):
    return Cg1[i]
model.Cg1 = Param(model.gens, initialize=Cg1_init)

def Cg2_init(model, i):
    return Cg2[i]
model.Cg2 = Param(model.gens, initialize=Cg2_init)

def CG_init(model, i, j):
    return CG[i,j]
model.CG = Param(model.buses, model.gens, initialize=CG_init)

In [8]:
# Define variables
def Pg_init(model, i):
    return Pg0[i]
model.Pg = Var(range(NG), within=PositiveReals, initialize=Pg_init)

def Pinj_init(model, i):
    return Pinj0[i]
model.Pinj = Var(range(NB), within=Reals, initialize=Pinj_init)

def Va_init(model, i):
    return Va0[i]
model.Va = Var(range(NB), within=Reals, initialize=Va_init)

model.dual = Suffix(direction=Suffix.IMPORT)

In [9]:
# Define objectives
def Gen_cost(model):
    cost0 = sum(model.Cg0[i] for i in model.gens)
    cost1 = sum(model.Cg1[i] * model.Pg[i] for i in model.gens)
    cost2 = sum(model.Cg2[i] * model.Pg[i]**2 for i in model.gens)
    return (cost0 + cost1 + cost2)
model.obj = Objective(rule=Gen_cost, sense=minimize)

In [10]:
# Define Constraints
def Pmax_rule(model, i):
    return model.Pg[i] <= model.Pmax[i]
model.c_Pmax = Constraint(model.gens, rule=Pmax_rule)

def Pmin_rule(model, i):
    return model.Pg[i] >= model.Pmin[i]
model.c_Pmin = Constraint(model.gens, rule=Pmin_rule)

def Lmax_rule(model, i):
    return sum(model.Bf[i,j]*model.Va[j] for j in model.buses) <= model.Lmax[i]
model.c_Lmax = Constraint(model.lines, rule=Lmax_rule)

def Lmin_rule(model, i):
    return sum(model.Bf[i,j]*model.Va[j] for j in model.buses) >= model.Lmin[i]
model.c_Lmin = Constraint(model.lines, rule=Lmin_rule)

def Pinj_rule(model, i):
    return model.Pinj[i] == (sum(model.CG[i,j]*model.Pg[j] for j in model.gens) - model.Pd[i])
model.c_Pinj = Constraint(model.buses, rule=Pinj_rule)

def PF_rule(model, i):
    return model.Pinj[i] == sum(model.B[i,j]*model.Va[j] for j in model.buses)
model.c_PF = Constraint(model.buses, rule=PF_rule)

In [11]:
# Solve the problem
solver = SolverFactory("mindtpy")
solver.solve(model)

In [12]:
model.display()

Model OPF

  Variables:
    Pg : Size=10, Index=Pg_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 :  6.608459942695003 :  None : False : False : PositiveReals
          1 :     0 :  6.460000062804398 :  None : False : False : PositiveReals
          2 :     0 :  6.608459942471459 :  None : False : False : PositiveReals
          3 :     0 :   6.52000005983922 :  None : False : False : PositiveReals
          4 :     0 : 5.0800000506333065 :  None : False : False : PositiveReals
          5 :     0 : 6.6084599418883965 :  None : False : False : PositiveReals
          6 :     0 :  5.800000057684317 :  None : False : False : PositiveReals
          7 :     0 :  5.640000056136645 :  None : False : False : PositiveReals
          8 :     0 :  6.608459942793593 :  None : False : False : PositiveReals
          9 :     0 :  6.608459943071543 :  None : False : False : PositiveReals
    Pinj : Size=39, Index=Pinj_index
        Key : Lower : 

In [25]:
# Print dual variables (shadow prices)
model.dual.pprint()

dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key        : Value
     c_Lmax[0] :  -5.173499417519515e-09
    c_Lmax[10] : -1.0418281750048098e-08
    c_Lmax[11] : -1.2989281905691557e-08
    c_Lmax[12] :  -5.897919730023587e-09
    c_Lmax[13] :  -2.086925136876417e-09
    c_Lmax[14] :   -8.13353602922655e-09
    c_Lmax[15] :   -6.91429127541709e-09
    c_Lmax[16] :  -6.853730421450688e-09
    c_Lmax[17] : -2.3298548775740888e-08
    c_Lmax[18] : -1.5844898008006302e-08
    c_Lmax[19] : -3.2590877988859266e-09
     c_Lmax[1] :  -7.119937510491267e-09
    c_Lmax[20] : -1.0189377784184044e-08
    c_Lmax[21] :  -9.987462167817988e-09
    c_Lmax[22] : -1.5398013473608167e-08
    c_Lmax[23] :  -8.505154514148243e-09
    c_Lmax[24] :  -5.540473465502049e-09
    c_Lmax[25] :  -1.430825971417092e-08
    c_Lmax[26] :  -4.705398501407228e-09
    c_Lmax[27] :  -5.342032374499292e-09
    c_Lmax[28] :  -7.725990221684723e-09
    c_Lmax[29] : -1.1627033938768815e-08
     c_Lmax[2] : -1.0

In [14]:
# Calculate average price
price = 0
for i in range(NB):
    price = price + model.dual[model.c_PF[i]]
price = price/NB/baseMVA
print("Average price = %.3f" %price)

Average price = 13.517


In [15]:
# Generation and cost
r_pg = np.zeros((NG, 1))
r_cost = np.zeros((NG, 1))
r_mc = np.zeros((NG, 1))

for i in range(NG):
    r_pg[i] = model.Pg[i]()
    r_cost[i] = Cg0[i] + Cg1[i]*r_pg[i] + Cg2[i]*r_pg[i]**2
    r_mc[i] = Cg1[i]/baseMVA + Cg2[i]*r_pg[i]/baseMVA

print("Pg = \n", r_pg)
print("Cost = \n", r_cost)
print("MC = \n", r_mc)

Pg = 
 [[6.60845994]
 [6.46000006]
 [6.60845994]
 [6.52000006]
 [5.08000005]
 [6.60845994]
 [5.80000006]
 [5.64000006]
 [6.60845994]
 [6.60845994]]
Cost = 
 [[4565.6280797 ]
 [4367.16008303]
 [4565.6280794 ]
 [4446.84007983]
 [2733.24005296]
 [4565.62807861]
 [3538.20006864]
 [3350.36006501]
 [4565.62807983]
 [4565.62808021]]
MC = 
 [[6.90845994]
 [6.76000006]
 [6.90845994]
 [6.82000006]
 [5.38000005]
 [6.90845994]
 [6.10000006]
 [5.94000006]
 [6.90845994]
 [6.90845994]]


In [16]:
print(model.obj())
print(np.sum(r_cost))

41263.9407472222
41263.9407472222


### PyPower solution

In [17]:
import pypower.api as pp
ppc = case39()
pp_dcopf = pp.rundcopf(ppc)
pp_bus = pp_dcopf["bus"]
pp_branch = pp_dcopf["branch"]
pp_gen = pp_dcopf["gen"]

PYPOWER Version 5.1.4, 27-June-2018 -- DC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 0.06 seconds
Objective Function Value = 41263.94 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             39     Total Gen Capacity    7367.0           0.0 to 0.0
Generators        10     On-line Capacity      7367.0           0.0 to 0.0
Committed Gens    10     Generation (actual)   6254.2               0.0
Loads             21     Load                  6254.2               0.0
  Fixed           21       Fixed               6254.2               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches          46     Losses (I^2 * Z)         0.

In [18]:
pp_pg = pp_gen[:, PG]/baseMVA
pp_va = pp_bus[:, VA]/180*np.pi
pp_pd = pp_bus[:, PD]/baseMVA
pp_pinj = (np.matmul(CG, pp_pg) - pp_pd)

In [19]:
# Power injection at buses
pp_pbus = np.matmul(B, pp_va)*baseMVA
print(pp_pbus)
pp_pinj = (np.matmul(CG, pp_pg) - pp_pd)*baseMVA
print(pp_pinj)

[[-9.76000000e+01 -5.24839748e-14 -3.22000000e+02 -5.00000000e+02
   1.81675080e-12 -1.93406730e-12 -2.33800000e+02 -5.22000000e+02
  -6.50000000e+00  1.49094301e-12 -5.73523412e-13 -8.53000000e+00
  -1.26461458e-13 -8.94720261e-13 -3.20000000e+02 -3.29000000e+02
   9.89100668e-13 -1.58000000e+02  2.15700133e-13 -6.80000000e+02
  -2.74000000e+02 -1.15204693e-13 -2.47500000e+02 -3.08600000e+02
  -2.24000000e+02 -1.39000000e+02 -2.81000000e+02 -2.06000000e+02
  -2.83500000e+02  6.60846000e+02  6.36800000e+02  6.60846000e+02
   6.52000000e+02  5.08000000e+02  6.60846000e+02  5.80000000e+02
   5.64000000e+02  6.60846000e+02 -4.43154000e+02]]
[ -97.6           0.         -322.         -500.            0.
    0.         -233.8        -522.           -6.5           0.
    0.           -8.53          0.            0.         -320.
 -329.            0.         -158.            0.         -680.
 -274.            0.         -247.5        -308.6        -224.
 -139.         -281.         -206.     

In [20]:
pp_cost = np.sum(Cg0) + np.sum(Cg1 * pp_pg) + np.sum(Cg2 * pp_pg**2)